# Experimenting with Graph Inputs to PyGCN 
## References: 

- [ ] https://github.com/tkipf/pygcn
- [ ] https://github.com/DaehanKim/vgae_pytorch
- [ ] https://arxiv.org/abs/1611.07308


## Objective

This notebook is going to explain how we process graph inputs for use with GAE model. 

## Input Data

We start with a citation network dataset (in this case, Cora). The dataset is just a list of publications with labels assigned based on what topic they cover. The dataset also includes information on which publications cite each other. From this, we can build a graph representing the citation network. 

The graph that represents this network is undirected (i.e. we assume that if node 1 is connected to node 2, then node 2 is also connected to node 1) and that it is unweighted (i.e. all the edges have the same weights). The graph is represented here as an adjacency matrix and a features matrix. Both of these matrices are sparse since each publication is only connected to a few other publications (generally). 

### Variables

- **N** : number of nodes in graph 
- **A** : adj matrix (size N x N) 
- **D** : degree matrix of A (i.e. a diagonal matrix where each entry represents the number of edges going into the node corresponding to that row) 
- **F** : number of features that describe each node 
- **X** : feature matrix (size N x F? paper says N x D but I don't know what D would be in this case?)
- **Z** : stochastic latent variables (size N x F) 

### Adj

The adjacency matrix contains 1's indicating where 2 publications cite each other. Kipf and Welling define the adj matrix as also having all diagonal elements set to 1 to indicate that nodes/publications are connected to themselves. This matrix is converted to $\tilde{A}$, which is a symmetrically normalized adjacency matrix computed as $\tilde{A} = D^{-\frac{1}{2}} A D^{-\frac{1}{2}}$. 

## Brainstorm: How to convert our data to graph representation?



## Utils

In [1]:
import numpy as np
import scipy.sparse as sp
import torch

In [2]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [3]:
def load_data(path="../data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    # ref: https://github.com/tkipf/pygcn/issues/3
    # adj is populated with 0's and 1's
    # if adj.T > adj then one of those entries is 0
    # therefore by multiplying them together, we get 0
    # the + (*) and - (*) terms where * = adj.T.multiply(adj.T > adj)
    # are for removing directed edges so that we are left with
    # an UNDIRECTED graph (i.e. edges are bidirectional)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [4]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [5]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [6]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [7]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [11]:
print("adj", adj)
print("adj type", type(adj))
print("adj[0]", adj[0])
# adj is a torch tensor in the sparse COO layout
# to create use torch.sparse.FloatTensor(indices, values, shape)
# adj is normalized!

adj tensor(indices=tensor([[   0,    8,   14,  ..., 1389, 2344, 2707],
                       [   0,    0,    0,  ..., 2707, 2707, 2707]]),
       values=tensor([0.1667, 0.1667, 0.0500,  ..., 0.2000, 0.5000, 0.2500]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)
adj type <class 'torch.Tensor'>
adj[0] tensor(indices=tensor([[  0,   8,  14, 258, 435, 544]]),
       values=tensor([0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667]),
       size=(2708,), nnz=6, layout=torch.sparse_coo)


In [14]:
print("features", features.shape)
# features is a torch array of size num_samples x num_features
# features are normalized!

features torch.Size([2708, 1433])


In [17]:
print("labels", labels.shape)
# labels is a torch array of size num_samples x 1

labels torch.Size([2708])


### TODO
- [ ] Are the GCN layers presented here the same as in the GVAE repo? 
- [ ] implement the GCN layers
- [ ] implement the encoder and decoder
- [ ] implement the training loop
- [ ] generate my own graphs

### Conceptual Questions: 
- [ ] this works on one large graph - how would I train on lots of smaller graphs? 
- [ ] why does gVAE worry about graph rotational invariance but I do not with GVAE? 